In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier
from fairGAN import Medgan
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
def train_fairgan(datapath):

    #data = np.load(datapath, allow_pickle = True)
    inputDim = data.shape[1]-1
    inputNum = data.shape[0]
    tf.reset_default_graph()
    mg = Medgan(dataType='count',
                inputDim=inputDim,
                embeddingDim=128,
                randomDim=128,
                generatorDims=(128,128),
                discriminatorDims=(256,128),
                compressDims=(),
                decompressDims=(),
                bnDecay=0.99,
                l2scale=0.001)

    model_file = ''
    out_file = 'fair'
    batch_size = 100
    
    mg.train(dataPath=datapath,
             modelPath=model_file,
             outPath=out_file,
             pretrainEpochs=200,
             nEpochs=1000,
             discriminatorTrainPeriod=2,
             generatorTrainPeriod=1,
             pretrainBatchSize=100,
             batchSize=batch_size,
             saveMaxKeep=0)
    tf.reset_default_graph()
    return mg.generateData(nSamples=inputNum,
                        modelFile='fair-2',
                        batchSize=batch_size,
                        outFile=out_file)

data = pd.read_csv("~/Desktop/causal_dp/data/census_data.csv")
print(len(data))

def label_fix(label):
    if label==' <=50K':
        return 0
    else:
        return 1
    
data['income_bracket'] = data['income_bracket'].apply(label_fix)



gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

feat_cols = [gender, occupation, marital_status, relationship, education, workclass, native_country,
            age, education_num, capital_gain, capital_loss, hours_per_week]
from sklearn.feature_selection import mutual_info_classif
print("Base MI RACE;INCOME = ", mutual_info_classif(np.expand_dims(data['income_bracket'], -1), data['race'], discrete_features = True))
print("Base MI Gender;INCOME = ", mutual_info_classif(np.expand_dims(data['income_bracket'], -1), data['gender'], discrete_features = True))



from sklearn import preprocessing
for feat in ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'gender', 'native_country']:
    data[feat] = preprocessing.LabelEncoder().fit_transform(data[feat])

data.drop(['capital_gain', 'capital_loss', 'education'], axis = 1, inplace = True)


#np.save('adult', data.values, allow_pickle = True)

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(data)
data[data.columns] = scaler.fit_transform(data)
pickle.dump(data.values, open( "adult.npy", "wb" ) )
print(train_fairgan('adult.npy'))

32561
Base MI RACE;INCOME =  [0.00580715]
Base MI Gender;INCOME =  [0.02576524]
adult.npy
Pretrain_Epoch:0, trainLoss:0.332826, validLoss:0.316264, validReverseLoss:0.000000
Pretrain_Epoch:1, trainLoss:0.314911, validLoss:0.297301, validReverseLoss:0.000000
Pretrain_Epoch:2, trainLoss:0.285666, validLoss:0.284637, validReverseLoss:0.000000
Pretrain_Epoch:3, trainLoss:0.283976, validLoss:0.284186, validReverseLoss:0.000000
Pretrain_Epoch:4, trainLoss:0.262841, validLoss:0.193588, validReverseLoss:0.000000
Pretrain_Epoch:5, trainLoss:0.190965, validLoss:0.190743, validReverseLoss:0.000000
Pretrain_Epoch:6, trainLoss:0.190493, validLoss:0.190776, validReverseLoss:0.000000
Pretrain_Epoch:7, trainLoss:0.190476, validLoss:0.190720, validReverseLoss:0.000000
Pretrain_Epoch:8, trainLoss:0.190478, validLoss:0.190671, validReverseLoss:0.000000
Pretrain_Epoch:9, trainLoss:0.190468, validLoss:0.190788, validReverseLoss:0.000000
Pretrain_Epoch:10, trainLoss:0.178903, validLoss:0.173513, validRevers

In [ ]:
data = np.load('adult_large.pkl', allow_pickle = True)

In [ ]:
data.values

In [ ]:
data